
# NYC Food Access — Geospatial Starter Notebook



In [24]:
import pandas as pd, json
from rich import print

In [7]:
CENSUS_TRACTS_PATH = "data/census_tracts.csv"   # e.g., 2020 Census tracts GeoJSON/GeoPackage/SHAPE
PANTRIES_PATH      = "data/pantry_locations.csv"    # can be a GeoJSON of points OR a CSV with lat/lon columns
OPTIONAL_NEIGHBORHOODS_PATH = None                      # e.g., neighborhoods or NTA boundaries (GeoJSON), or None
SUPPLY_GAP_CSV     = "data/Emergency_Food_Supply_Gap_20251110.csv"                    # e.g., "data/supply_gap_by_neighborhood.csv" or None


In [8]:
tracts = pd.read_csv(CENSUS_TRACTS_PATH)
supply_gap = pd.read_csv(SUPPLY_GAP_CSV)

In [9]:
# Rename for consistency
supply_gap = supply_gap.rename(columns={
    "Neighborhood Tabulation Area NTA)": "nta2020",
    "Neighborhood Tabulation Area (NTA) Name": "ntaname"
})

# Strip spaces / make uppercase (to ensure a clean join)
supply_gap["nta2020"] = supply_gap["nta2020"].str.strip().str.upper()
tracts["nta2020"] = tracts["nta2020"].astype(str).str.strip().str.upper()

In [15]:
supply_gap.head()

,Year,nta2020,ntaname,Supply Gap (lbs.),Food Insecure Percentage,Unemployment Rate,Vulnerable Population Score,Weighted Score,Rank
0,2025,BX0401,Concourse-Concourse Village,"-102,142.62426294",26.29%,1053.82%,0.57,5.133096,161.0
1,2025,BX0303,Crotona Park East,"-333,492.64867191",27.24%,1308.90%,0.55,5.628794,141.0
2,2025,MN1102,East Harlem (North),"527,499.52610583",26.68%,1055.74%,0.47,5.908840,121.0
3,2025,BX0602,Tremont,"-113,652.81465869",28.96%,1214.11%,0.53,5.711963,134.0
4,2025,BK1204,Mapleton-Midwood (West),"451,455.520860002",16.87%,564.74%,0.61,5.731608,132.0


In [18]:
merged = tracts.merge(supply_gap, on="nta2020", how="left")

In [ ]:
merged.dropna(subset=["Supply Gap (lbs.)"], inplace=True)

,geoid,ctlabel,ct2020,boroct2020,borocode,boroname,ntaname_x,nta2020,cdeligibil,cdta2020,...,centroid_lat,centroid_lon,Year,ntaname_y,Supply Gap (lbs.),Food Insecure Percentage,Unemployment Rate,Vulnerable Population Score,Weighted Score,Rank
1,36061001401,14.01,1401,1001401,1,Manhattan,Lower East Side,MN0302,I,MN03,...,40.714983,-73.986699,2025.0,Lower East Side,"740,167.495815",20.96%,647.31%,0.47,6.194972,90.0
2,36061001401,14.01,1401,1001401,1,Manhattan,Lower East Side,MN0302,I,MN03,...,40.714983,-73.986699,2024.0,Lower East Side,"815,973.731548145",23.39%,5.32%,0.13,5.767978,72.0
3,36061001401,14.01,1401,1001401,1,Manhattan,Lower East Side,MN0302,I,MN03,...,40.714983,-73.986699,2023.0,Lower East Side,"290,239.4793",18.46%,5.55%,0.13,5.293685,102.0
4,36061001402,14.02,1402,1001402,1,Manhattan,Lower East Side,MN0302,E,MN03,...,40.717464,-73.985782,2025.0,Lower East Side,"740,167.495815",20.96%,647.31%,0.47,6.194972,90.0
5,36061001402,14.02,1402,1001402,1,Manhattan,Lower East Side,MN0302,E,MN03,...,40.717464,-73.985782,2024.0,Lower East Side,"815,973.731548145",23.39%,5.32%,0.13,5.767978,72.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6828,36061003200,32.00,3200,1003200,1,Manhattan,East Village,MN0303,I,MN03,...,40.725531,-73.984039,2024.0,East Village,"737,116.123057523",15.86%,5.34%,0.12,5.307906,114.0
6829,36061003200,32.00,3200,1003200,1,Manhattan,East Village,MN0303,I,MN03,...,40.725531,-73.984039,2023.0,East Village,"842,491.8974",15.03%,5.53%,0.11,5.446167,91.0
6830,36061000202,2.02,202,1000202,1,Manhattan,Lower East Side,MN0302,E,MN03,...,40.711168,-73.980467,2025.0,Lower East Side,"740,167.495815",20.96%,647.31%,0.47,6.194972,90.0
6831,36061000202,2.02,202,1000202,1,Manhattan,Lower East Side,MN0302,E,MN03,...,40.711168,-73.980467,2024.0,Lower East Side,"815,973.731548145",23.39%,5.32%,0.13,5.767978,72.0


In [27]:
merged.to_csv("data/neighborhood_supply_gap.csv", index=False)